In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
import tensorflow as tf
import random
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Carregando os dados de treinamento e teste
train_df = pd.read_csv("train_audience.csv")  # Carregando dados de treino
train_df = train_df.groupby('Rating', group_keys=False).apply(lambda x: x.sample(1500)).reset_index(drop=True)
vectorizer = TfidfVectorizer(stop_words='english') 
reviews = vectorizer.fit_transform(train_df['Review'])
rating = train_df['Rating']

In [ ]:

x_train, x_val, y_train, y_val = train_test_split (reviews, rating,test_size = 0.2)

In [ ]:

from sklearn.preprocessing import LabelEncoder


y_train_str = y_train.astype(str)
y_val_str = y_val.astype(str)

# for val in y_train:
#     y_train_str.append(str(val))

# for val in y_val:
#     y_val_str.append(str(val))


label_enc = LabelEncoder()
ye_train = label_enc.fit_transform(y_train_str)
ye_val = label_enc.transform(y_val_str)

In [ ]:
from keras.utils import to_categorical

y_ohe_train = to_categorical(ye_train, num_classes=10)
y_ohe_val = to_categorical(ye_val, num_classes=10)

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns


def evaluate_metrics(y, p):
    # Cálculo das métricas de acerto.
    print('Accuracy:', round(metrics.accuracy_score(y, p),3))
    print('Accuracy (balanced):', round(metrics.balanced_accuracy_score(y, p), 3))

    # Calculo da matriz de confusão.
    c = metrics.confusion_matrix(y, p)
    r = c / c.sum(axis=1, keepdims=True)

    plt.figure(figsize=(10, 8))
    sns.heatmap(r, annot=True, fmt=".2f", cmap="Blues")
    plt.xlabel("Classe predita")
    plt.ylabel("Classe verdadeira")
    plt.title("Matriz de confusão relativa")
    plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rndForest = RandomForestClassifier(max_depth=8, n_estimators=800, max_features=800)
rndForest.fit(x_train, y_ohe_train)
p_val = rndForest.predict(x_val)

In [ ]:

predictions = np.argmax(p_val, axis=1)

evaluate_metrics(ye_val, predictions)